In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
igfold = 0
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
import cudf
import numpy as np
import gc

import glob

In [3]:
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [1, 2, 3],
    'type_mp': [1, 6, 3]
})


In [4]:
def calc_visits(df):
    df = df.merge(
        df_type,
        how='left',
        on='type'
    )
    df['session'] = df['session'].astype('int32')
    df['aid'] = df['aid'].astype('int32')
    df.ts = (df.ts/1000).astype('int32')
    df.drop(['type'], axis=1, inplace=True)
    df = df.rename(columns={'type_': 'type'})
    df = df.merge(
        df[['session']].groupby(['session']).size().reset_index().rename(columns={0: 'session_len'}),
        how='left',
        on='session'
    )
    df = df.merge(
        df[['session', 'ts']].groupby(['session']).max().reset_index().rename(columns={'ts': 'ts_max'}),
        how='left',
        on='session'
    )
    df = df.sort_values(['session', 'ts'], ascending=[True, False])
    df['dummy'] = 1
    df['rank'] = df.groupby(['session']).dummy.cumsum()
    df.drop(['dummy'], axis=1, inplace=True)
    min_val = (2 ** 0.1-1)
    max_val = (2 ** 1-1)
    df['wgt_1'] = (min_val+(max_val-min_val)*(df['rank']-1)/(df['session_len']))*df['type_mp']
    min_val = (2 ** 0.5-1)
    max_val = (2 ** 1-1)
    df['wgt_2'] = (min_val+(max_val-min_val)*(df['rank']-1)/(df['session_len']))*df['type_mp']
    df['ts_diff'] = df['ts_max']-df['ts']
    df['clicks'] = (df['type']==1).astype('int')
    df['carts'] = (df['type']==2).astype('int')
    df['orders'] = (df['type']==3).astype('int')
    df = df[
        ['session', 'aid', 'ts_diff', 'rank', 'wgt_1', 'wgt_2', 'clicks', 'carts', 'orders']
    ].groupby([
        'session', 'aid',
    ]).agg({
        'ts_diff': 'min',
        'rank': 'min',
        'wgt_1': 'sum',
        'wgt_2': 'sum',
        'clicks': 'sum',
        'carts': 'sum',
        'orders': 'sum'
    }).reset_index().rename(columns={
        'rank': 'rev_order', 'wgt_1': 'pv_wgt_1', 'wgt_2': 'pv_wgt_2', 'aid': 'cand'
    })
    return(df)

In [5]:
!mkdir -p ./data/candidates/train/pv/

In [6]:
df = cudf.read_parquet('./data/xgb_train_x.parquet')

In [7]:
df = calc_visits(df)

In [8]:
df['pv_wgt_1'] = df['pv_wgt_1'].astype('float32')
df['pv_wgt_2'] = df['pv_wgt_2'].astype('float32')
df['rev_order'] = df['rev_order'].astype('int16')
df['clicks'] = df['clicks'].astype('int16')
df['carts'] = df['carts'].astype('int16')
df['orders'] = df['orders'].astype('int16')

In [9]:
df.to_parquet('./data/candidates/train/pv/cand.parquet')

In [10]:
!mkdir -p ./data/candidates/sub/pv/

In [11]:
df = cudf.read_parquet('../../data/test.parquet')

In [12]:
df = calc_visits(df)

In [13]:
df['pv_wgt_1'] = df['pv_wgt_1'].astype('float32')
df['pv_wgt_2'] = df['pv_wgt_2'].astype('float32')
df['rev_order'] = df['rev_order'].astype('int16')
df['clicks'] = df['clicks'].astype('int16')
df['carts'] = df['carts'].astype('int16')
df['orders'] = df['orders'].astype('int16')

In [14]:
df.to_parquet('./data/candidates/sub/pv/cand.parquet')

In [15]:
!cp -r ./data/candidates/train/pv/ ./data_folds/fold_0/candidates/train/pv/
!cp -r ./data/candidates/train/pv/ ./data_folds/fold_1/candidates/train/pv/
!cp -r ./data/candidates/train/pv/ ./data_folds/fold_2/candidates/train/pv/
!cp -r ./data/candidates/train/pv/ ./data_folds/fold_3/candidates/train/pv/
!cp -r ./data/candidates/train/pv/ ./data_folds/fold_4/candidates/train/pv/

In [16]:
!cp -r ./data/candidates/sub/pv/ ./data_folds/fold_0/candidates/sub/pv/
!cp -r ./data/candidates/sub/pv/ ./data_folds/fold_1/candidates/sub/pv/
!cp -r ./data/candidates/sub/pv/ ./data_folds/fold_2/candidates/sub/pv/
!cp -r ./data/candidates/sub/pv/ ./data_folds/fold_3/candidates/sub/pv/
!cp -r ./data/candidates/sub/pv/ ./data_folds/fold_4/candidates/sub/pv/